In [ ]:
# # Step 1: Install the package
!pip install git+https://github.com/smazzanti/mrmr


In [ ]:
# Step 2: Load packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from mrmr import mrmr_classif

sns.set_style("whitegrid")

import warnings
warnings.filterwarnings("ignore")


# Connect to google drive 
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Step 3: Load data
file1 = pd.read_csv('/content/drive/MyDrive/Data_Tutorial_BC/data_clinical_patient.csv')
file2 = pd.read_csv('/content/drive/MyDrive/Data_Tutorial_BC/data_mRNA_median_all_sample_Zscores.csv')

In [ ]:
# Have a quick look at data
file2.info()
file2.head()

In [ ]:
# Drop unused column
file2 = file2.drop('Entrez_Gene_Id', axis=1)

# Drop NA in GeneID
file2 = file2[file2['Hugo_Symbol'].notna()]

# Check null in GeneID columns
file2['Hugo_Symbol'].isnull().sum()

# Check duplicated values
print('The number of duplicated values of Hugo_Symbol in data:', file2['Hugo_Symbol'].duplicated().sum())

# Drop duplicated values for Gene ID
file2 = file2.drop_duplicates(subset=['Hugo_Symbol'])
print('After pre-processing, the number of duplicated values of Hugo_Symbol in data:',
     file2['Hugo_Symbol'].duplicated().sum())
print('Shape of Gene data:', file2.shape)

# Tranpose patient ID  to rows in order to match two data
file2 = file2.set_index('Hugo_Symbol').T.rename_axis('PATIENT_ID').rename_axis(None, axis=1).reset_index()
print('New shape of Gene data:', file2.shape)
file2.head(3)

In [ ]:
# Merge gene data with OS time and status
data = pd.merge(file1[['PATIENT_ID','OS_MONTHS','OS_STATUS']],file2, how="inner", on=["PATIENT_ID"])

In [ ]:
# Have a quick look at data
data.head()

In [ ]:
# Step 4: Preprocess data & Explore data

# 4.1 Check duplicated values
print('The number of duplicated values in data', data.duplicated().sum())
print('The shape of data is', data.shape)

# Step 4.2: Deal with missing values

# Check missing values
print('Total missing value in the dataset:', data.isnull().sum().sum())
cols_missvalue = data.columns[data.isnull().sum()>0]
print('List columns having missing data:', cols_missvalue)

# Replace missing values with average values
data[cols_missvalue] = data[cols_missvalue].fillna(data[cols_missvalue].mean())

# Check missing values again
print('After preprocessing, the number of missing values:', data.isna().sum().sum())


In [ ]:
# Step 5: Feature extraction

# 5.1: Normalise data
ss = MinMaxScaler()
X_norm = data.drop(['OS_STATUS', 'OS_MONTHS','PATIENT_ID'], axis = 1)
X_norm = pd.DataFrame(ss.fit_transform(X_norm), columns=X_norm.columns)


# 5.2: Select feature using MRMR
y_mrmr = data['OS_MONTHS']

features_selected = mrmr_classif(X_norm, y_mrmr, K = 50)
X_mrmr = data[features_selected]

In [ ]:
# 5.3 Save to csv file
df_mrmr = X_mrmr
df_mrmr['PATIENT_ID'] = data['PATIENT_ID']
df_mrmr.to_csv('/content/drive/MyDrive/Data_Tutorial_BC/Gene_MRMR_50.csv', index=False)

In [ ]:
df_mrmr.shape